In [5]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.request import URLError
from copy import deepcopy
import numpy as np
import pickle
import time
import undetected_chromedriver as uc

In [69]:
# Collecting all market_report_links
driver = uc.Chrome()
driver.get("https://www.alliedmarketresearch.com/reports-store")

In [2]:
market_report_links = []
with open("market_report_links.pkl", "rb") as f:
    market_report_links = pickle.load(f)

In [70]:
count = 1
while True:
    print(count)
    market_report_elements = driver.find_elements(By.CLASS_NAME, "reportpg-sec")
    for market_report_element in market_report_elements:
        media_element = market_report_element.find_element(By.CLASS_NAME, "media-body")
        market_report_links.append(media_element.find_element(By.XPATH, ".//h3/a").get_attribute("href"))
    pagination_element = driver.find_element(By.CLASS_NAME, "pagination")
    page_elements = pagination_element.find_elements(By.CLASS_NAME, "page-item")
    if page_elements[-1].text != "Next":
        break
    current_url = driver.current_url
    page_elements[-1].click()
    WebDriverWait(driver, 10000).until(EC.url_changes(current_url))
    count += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [74]:
with open("market_report_links.pkl", "wb") as f:
    pickle.dump(market_report_links, f)

In [ ]:
# Collecting all soups fro market_report_links

In [3]:
from bs4 import BeautifulSoup
import requests


In [4]:
def get_cagr_from_str(line):
    return float(line.split("%")[0].split()[-1])

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

i = 31087

cached_soups = set(market_report_links[:i])


In [8]:
while i < len(market_report_links):
    start = time.time()
    url = market_report_links[i]
    if url in cached_soups:
        print(i, "repeated url")
        i += 1
        continue
    response = None
    while not response or response.status_code != 200:
        response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    with open("cached_stocks_soups/{}.pkl".format(url.split('/')[-1]), "wb") as f:
        pickle.dump(soup, f)
    end = time.time()
    print(i, end - start)
    i += 1

31087 8.769974946975708
31088 3.667480707168579
31089 10.541262149810791
31090 4.779137134552002
31091 3.6875462532043457
31092 6.99338698387146
31093 6.4308178424835205
31094 3.5831968784332275
31095 5.675045967102051
31096 9.327256202697754
31097 3.7686269283294678
31098 3.003497838973999
31099 4.507274866104126
31100 53.5613911151886
31101 repeated url
31102 repeated url
31103 repeated url
31104 repeated url
31105 repeated url
31106 repeated url
31107 repeated url
31108 repeated url
31109 repeated url
31110 repeated url
31111 repeated url
31112 repeated url
31113 repeated url
31114 repeated url
31115 repeated url
31116 repeated url
31117 repeated url
31118 repeated url
31119 repeated url
31120 repeated url
31121 repeated url
31122 repeated url
31123 repeated url
31124 repeated url
31125 1.2005410194396973
31126 1.3582489490509033
31127 1.2309000492095947
31128 1.3329758644104004
31129 1.3725121021270752
31130 1.3909540176391602
31131 1.2685949802398682
31132 1.285714864730835
31133 

31425 1.3456850051879883
31426 1.3487298488616943
31427 1.5178091526031494
31428 1.619887113571167
31429 1.487779140472412
31430 1.3397929668426514
31431 1.381498098373413
31432 1.4490787982940674
31433 1.524914026260376
31434 1.4282941818237305
31435 1.3302228450775146
31436 1.2349979877471924
31437 1.446376085281372
31438 1.4119789600372314
31439 1.2521231174468994
31440 1.4202780723571777
31441 1.4381799697875977
31442 1.4126429557800293
31443 1.454185962677002
31444 1.4335558414459229
31445 1.3855609893798828
31446 1.3534362316131592
31447 1.6688249111175537
31448 1.3148469924926758
31449 1.317681074142456
31450 1.3394908905029297
31451 1.3343629837036133
31452 1.3281750679016113
31453 1.3585960865020752
31454 1.300887107849121
31455 1.3378520011901855
31456 1.213028907775879
31457 1.3298990726470947
31458 1.3422937393188477
31459 1.3436920642852783
31460 1.312248706817627
31461 1.3300862312316895
31462 1.3268380165100098
31463 1.3489117622375488
31464 1.39131498336792
31465 1.5946

31756 1.4269177913665771
31757 1.4112188816070557
31758 1.3255231380462646
31759 1.6602487564086914
31760 1.4006671905517578
31761 1.3484101295471191
31762 1.3487439155578613
31763 1.426973819732666
31764 1.334001064300537
31765 1.423356056213379
31766 1.466862678527832
31767 1.338352918624878
31768 1.3967978954315186
31769 1.5828769207000732
31770 1.5049340724945068
31771 1.3184318542480469
31772 1.4147651195526123
31773 1.3412117958068848
31774 1.4362199306488037
31775 1.4452688694000244
31776 1.433635950088501
31777 1.579252004623413
31778 1.4793651103973389
31779 1.4369308948516846
31780 1.54836106300354
31781 1.420335054397583
31782 1.4248037338256836
31783 1.4490339756011963
31784 1.5460920333862305
31785 1.5565738677978516
31786 1.3780522346496582
31787 1.3771100044250488
31788 1.3193929195404053
31789 1.5205700397491455
31790 1.3341100215911865
31791 1.3238952159881592
31792 1.3348979949951172
31793 1.5501198768615723
31794 1.361321210861206
31795 1.4004559516906738
31796 1.352

32088 1.4651658535003662
32089 1.4788947105407715
32090 1.4734828472137451
32091 1.4248011112213135
32092 1.5642070770263672
32093 1.3630881309509277
32094 1.3491687774658203
32095 1.3754918575286865
32096 1.37636399269104
32097 1.3275270462036133
32098 1.5558640956878662
32099 1.3287789821624756
32100 1.4091548919677734
32101 1.252082109451294
32102 1.3283720016479492
32103 1.3192472457885742
32104 1.2787001132965088
32105 1.2713677883148193
32106 1.3545811176300049
32107 1.3160197734832764
32108 1.325789213180542
32109 1.2534599304199219
32110 1.4263079166412354
32111 1.3202829360961914
32112 1.4849588871002197
32113 1.5002381801605225
32114 1.3304376602172852
32115 1.3154661655426025
32116 1.3435709476470947
32117 1.3322827816009521
32118 1.315039873123169
32119 1.2329058647155762
32120 1.384355068206787
32121 1.3911209106445312
32122 1.6321640014648438
32123 1.7702240943908691
32124 1.262239933013916
32125 1.3646798133850098
32126 1.3216891288757324
32127 1.2519819736480713
32128 1

32419 1.6858458518981934
32420 1.735579013824463
32421 1.528144121170044
32422 1.4931941032409668
32423 1.5206758975982666
32424 1.3372621536254883
32425 1.532792091369629
32426 1.412118911743164
32427 1.6464300155639648
32428 1.9889461994171143
32429 1.433211088180542
32430 1.4912261962890625
32431 1.6307330131530762
32432 1.4590742588043213
32433 1.4464750289916992
32434 2.2185120582580566
32435 2.344865083694458
32436 1.3448209762573242
32437 1.3357739448547363
32438 1.3682928085327148
32439 1.3738508224487305
32440 1.347602128982544
32441 1.3306729793548584
32442 1.323448896408081
32443 1.423095941543579
32444 1.250633955001831
32445 1.3487980365753174
32446 1.3445160388946533
32447 1.602612018585205
32448 1.3340418338775635
32449 1.9298551082611084
32450 1.3255927562713623
32451 1.3521637916564941
32452 1.2429440021514893
32453 2.136795997619629
32454 1.2762818336486816
32455 1.490753173828125
32456 2.0367040634155273
32457 1.437147855758667
32458 1.3386890888214111
32459 1.519322

...

61264 1.4709298610687256
61265 1.3813388347625732
61266 1.2280030250549316
61267 1.361335039138794
61268 1.3992218971252441
61269 1.360154151916504
61270 1.3416111469268799
61271 1.399505853652954
61272 2.2161989212036133
61273 1.3636460304260254
61274 1.3308379650115967
61275 1.2586238384246826
61276 1.3142969608306885
61277 1.414720058441162
61278 1.463996171951294
61279 1.4067280292510986
61280 1.3558459281921387
61281 1.2958340644836426
61282 1.353036880493164
61283 1.3516473770141602
61284 1.3368642330169678
61285 1.554175853729248
61286 1.7969038486480713
61287 2.0716841220855713
61288 1.8270602226257324
61289 1.5886430740356445
61290 1.259774923324585
61291 1.3327867984771729
61292 1.4867439270019531
61293 1.497385025024414
61294 1.4361028671264648
61295 1.317742109298706
61296 1.341231107711792
61297 1.4309189319610596
61298 1.3829920291900635
61299 1.2475390434265137
61300 1.3446300029754639
61301 1.8309080600738525
61302 1.3766629695892334
61303 1.3318219184875488
61304 1.405

61596 1.276048183441162
61597 1.2825291156768799
61598 1.598677158355713
61599 1.5836448669433594
61600 1.3194129467010498
61601 2.376365900039673
61602 1.5195508003234863
61603 1.3315658569335938
61604 1.3315708637237549
61605 1.630972146987915
61606 1.3888018131256104
61607 1.2777831554412842
61608 1.3464901447296143
61609 1.3197293281555176
61610 1.3346481323242188
61611 1.3462250232696533
61612 1.3291568756103516
61613 1.317875862121582
61614 1.2253930568695068
61615 1.4740562438964844
61616 1.291862964630127
61617 1.2523062229156494
61618 1.3024280071258545
61619 1.3348579406738281
61620 1.3296291828155518
61621 1.315464973449707
61622 1.4483659267425537
61623 1.4917631149291992
61624 1.277519702911377
61625 1.4260706901550293
61626 1.432502031326294
61627 1.230309247970581
61628 1.3344988822937012
61629 1.3289690017700195
61630 1.3296217918395996
61631 1.482964277267456
61632 1.3013181686401367
61633 1.3121778964996338
61634 1.244757890701294
61635 1.4185998439788818
61636 1.2274

61927 1.3455240726470947
61928 1.317784070968628
61929 1.3311421871185303
61930 1.349320888519287
61931 1.4045257568359375
61932 1.3287019729614258
61933 1.4378459453582764
61934 1.6126940250396729
61935 1.47176194190979
61936 1.3249499797821045
61937 1.43206787109375
61938 1.4030330181121826
61939 1.3550770282745361
61940 1.2322697639465332
61941 1.4425468444824219
61942 1.4665648937225342
61943 1.2924830913543701
61944 1.3307793140411377
61945 1.3464577198028564
61946 1.5224058628082275
61947 1.3766810894012451
61948 1.583380937576294
61949 1.4382500648498535
61950 1.3279619216918945
61951 1.334273099899292
61952 1.9514508247375488
61953 1.3250281810760498
61954 1.5377850532531738
61955 1.4960219860076904
61956 1.407372236251831
61957 1.3995399475097656
61958 1.3282458782196045
61959 1.3472330570220947
61960 1.4160819053649902
61961 1.3363170623779297
61962 1.3287830352783203
61963 1.4814152717590332
61964 1.2846629619598389
61965 1.5359361171722412
61966 1.4516429901123047
61967 1.4

62259 1.2661798000335693
62260 1.470069169998169
62261 1.3608200550079346
62262 1.3226051330566406
62263 1.2908380031585693
62264 1.3623392581939697
62265 1.4427177906036377
62266 1.323779821395874
62267 1.3223679065704346
62268 1.4311811923980713
62269 1.4415197372436523
62270 1.314028024673462
62271 1.4933340549468994
62272 1.408930778503418
62273 1.3190820217132568
62274 1.3308238983154297
62275 1.3157060146331787
62276 1.4645519256591797
62277 1.4053239822387695
62278 1.5495729446411133
62279 1.4289820194244385
62280 1.452897071838379
62281 1.165802001953125
62282 1.1541311740875244
62283 1.2021689414978027
62284 1.197300910949707
62285 1.4017889499664307
62286 1.287848949432373
62287 1.231109857559204
62288 1.2767610549926758
62289 1.1800270080566406
62290 1.1202588081359863
62291 1.128391981124878
62292 1.2395329475402832
62293 1.2188341617584229
62294 1.2293851375579834
62295 1.2269139289855957
62296 1.3796720504760742
62297 1.2817847728729248
62298 1.2332501411437988
62299 1.23

62590 1.2735681533813477
62591 1.2736070156097412
62592 1.3349170684814453
62593 1.33799409866333
62594 1.4386780261993408
62595 1.3270132541656494
62596 1.4772071838378906
62597 1.3029789924621582
62598 1.5211567878723145
62599 1.3326270580291748
62600 1.2526659965515137
62601 1.6216299533843994
62602 1.957448959350586
62603 1.7477219104766846
62604 1.679565191268921
62605 1.2624492645263672
62606 1.3264050483703613
62607 1.344292163848877
62608 1.3270421028137207
62609 1.3242461681365967
62610 1.4267160892486572
62611 1.4442110061645508
62612 1.2158491611480713
62613 1.3928790092468262
62614 1.377011775970459
62615 1.3350508213043213
62616 1.3273251056671143
62617 1.443120002746582
62618 1.4444990158081055
62619 1.3179349899291992
62620 1.3218519687652588
62621 1.387725830078125
62622 1.3786187171936035
62623 1.282759189605713
62624 1.3797330856323242
62625 1.4311907291412354
62626 1.3657941818237305
62627 1.3945679664611816
62628 1.333178997039795
62629 1.465252161026001
62630 1.455

In [18]:
# ignore_words = set()
# ignore_words.add("inc")
# ignore_words.add("inc.")
# ignore_words.add("corporation")
# ignore_words.add("se")
# ignore_words.add("ag")
# ignore_words.add("ltd")
# ignore_words.add("llc")
# ignore_words.add("se.")
# ignore_words.add("ag.")
# ignore_words.add("ltd.")
# ignore_words.add("llc.")
# ignore_words.add("systems")
# ignore_words.add("gmbh")
# ignore_words.add("gmbh.")
# ignore_words.add("technologies")
# ignore_words.add("co")
# ignore_words.add("co.")
# ignore_words.add("company")
# ignore_words.add("ag")
# ignore_words.add("n.v")
# ignore_words.add("ag.")
# ignore_words.add("n.v.")
# ignore_words.add("incorporated")
# ignore_words.add("limited")
# ignore_words.add("plc")
# ignore_words.add("lp")
# ignore_words.add("kgaa")
# ignore_words.add("plc.")
# ignore_words.add("lp.")
# ignore_words.add("kgaa.")
# ignore_words.add("group")
# ignore_words.add("a/s")
# ignore_words.add("emc")
# ignore_words.add("emc.")
# ignore_words.add("&")
# ignore_words.add("and")
# ignore_words.add("holdings")
# ignore_words.add("ab")
# ignore_words.add("kg")
# ignore_words.add("corp")
# ignore_words.add("s.p.a")
# ignore_words.add("s.a")
# ignore_words.add("ab.")
# ignore_words.add("kg.")
# ignore_words.add("corp.")
# ignore_words.add("s.p.a.")
# ignore_words.add("s.a.")
# ignore_words.add("industries")
# ignore_words.add("pvt")
# ignore_words.add("pvt.")
# ignore_words.add("private")

# def sanitize(company):
#     company_str_split = company.lower().strip().strip(".").split()
#     new_str = []
#     for word in company_str_split:
#         if word not in ignore_words:
#             new_str.append(word)
#     return_str = " ".join(new_str)
#     return return_str

In [1]:
from os import listdir
import numpy as np

In [2]:
files = [f for f in listdir("cached_stocks_soups")]

In [160]:
cagr_present_cases = []
special_cases = []

In [161]:
to_sift = files
start = time.time()
for i in range(len(to_sift)):
    if i % 100 == 0:
        print(i, time.time() - start)
        start = time.time()
    if to_sift[i][-4:] != ".pkl":
        continue
    soup = None
    with open("cached_stocks_soups/" + to_sift[i], "rb") as f:
        soup = pickle.load(f)
    text_body_element = soup.find(id="roOne")
    if not text_body_element:
        continue
    lines = text_body_element.findChildren("p")
    found = False
    for j in range(min(5, len(lines))):
        if "cagr" in lines[j].text.lower() and "%" in lines[j].text and not "cagr %" in lines[j].text.lower():
            found = True
            break
    if not found:
        lines = text_body_element.findChildren("h2")
        if len(lines) != 0 and "cagr" in lines[0].text.lower() and "%" in lines[0].text and not "cagr %" in lines[0].text.lower():
            found = True
    if found:
        cagr_present_cases.append(to_sift[i])
    else:
        special_cases.append(to_sift[i])

0 0.0007469654083251953
100 7.8132100105285645
200 7.8950889110565186
300 7.33054518699646
400 7.2196269035339355
500 7.310909986495972
600 7.438401699066162
700 7.361382007598877
800 7.137955188751221
900 7.245526075363159
1000 7.1397178173065186
1100 7.6648030281066895
1200 7.473712921142578
1300 7.7224743366241455
1400 8.524703979492188
1500 9.058245182037354
1600 7.788264989852905
1700 8.636500120162964
1800 7.395606994628906
1900 7.8855249881744385
2000 8.432502031326294
2100 9.21059513092041
2200 8.391440868377686
2300 7.451709985733032
2400 7.631505012512207
2500 7.497384309768677
2600 7.106580972671509
2700 7.04026985168457
2800 7.491776943206787
2900 7.329716920852661
3000 7.435449123382568
3100 8.265107154846191
3200 7.394715070724487
3300 7.085035085678101
3400 7.1053080558776855
3500 7.023916006088257
3600 7.044568061828613
3700 7.182348012924194
3800 7.082320690155029
3900 7.9688756465911865
4000 9.470296144485474
4100 9.23253607749939
4200 9.95533800125122
4300 12.2568061

34600 7.4606499671936035
34700 7.527982234954834
34800 7.458928108215332
34900 7.365733861923218
35000 7.339608907699585
35100 7.3961029052734375
35200 7.360492706298828
35300 7.481229782104492
35400 7.28135085105896
35500 7.293042898178101
35600 7.263607978820801
35700 7.316712856292725
35800 7.207508087158203
35900 7.275016784667969
36000 7.3012800216674805
36100 7.257104873657227
36200 7.28998589515686
36300 7.405163049697876
36400 7.4607532024383545
36500 7.401954650878906
36600 7.323040008544922
36700 7.469451904296875
36800 7.395946979522705
36900 7.328297853469849
37000 7.416295766830444
37100 7.831279039382935
37200 7.4170308113098145
37300 7.347798109054565
37400 7.220738887786865
37500 7.2430198192596436
37600 7.194688081741333
37700 7.2228569984436035
37800 7.381901025772095
37900 7.315256118774414
38000 7.562615871429443
38100 7.382475852966309
38200 7.342181205749512
38300 7.271753787994385
38400 7.363192081451416
38500 7.268635272979736
38600 7.295304298400879
38700 7.267

In [163]:
len(cagr_present_cases)

4717

In [164]:
cagr_present_cases

['airport-stands-equipment-market.pkl',
 'vcsel-for-data-communication-market-A06645.pkl',
 'ergonomic-chair-market-A12927.pkl',
 'anti-counterfeit-clothing-and-accessories-packaging-market.pkl',
 'iot-for-public-safety-market-A08798.pkl',
 'high-tibial-osteotomy-hto-plates-market.pkl',
 'eye-drop-and-lubricants-market.pkl',
 'catechin-market-A06590.pkl',
 'refinery-vacuum-distillation-units-market-A14251.pkl',
 'metal-packaging-market.pkl',
 'healthcare-chatbots-market.pkl',
 'radiodermatitis-market.pkl',
 'anti-blue-ray-myopia-lenses-market-A17385.pkl',
 'motorhome-vehicle-market-A07878.pkl',
 'hpv-testing-and-pap-test-market-A14373.pkl',
 'kids-toys-market-A06531.pkl',
 '4g-equipment-market.pkl',
 'micro-mobility-market-A11372.pkl',
 'mobile-bi-market.pkl',
 'veterinary-dental-equipment-market-A08434.pkl',
 'network-security-market-A10606.pkl',
 'translucent-concrete-market-A06302.pkl',
 'agrochemicals-market.pkl',
 'contraceptive-drugs-market-A09485.pkl',
 'florida-virtual-training

In [151]:
soup = None
with open("cached_stocks_soups/" + 'europe-gene-editing-market-A22058.pkl', "rb") as f:
    soup = pickle.load(f)
text_body_element = soup.find(id="roOne")
lines = text_body_element.findChildren("p")

In [152]:
found = False
for j in range(len(lines)):
    if "cagr" in lines[j].text.lower() and "%" in lines[j].text:
        found = True
        break

In [153]:
lines[j].text

'COVID-19 impact and revenue opportunity to provide benefits to industry stakeholders with the critical insights to assess the Europe gene editing market.Report Description: The report will cover market trends, size & revenue growth, segmentation, country analysis, competitive landscape, company market shares, and development strategies for this market. The market overview section of the report will cover all aspects of the market in terms of qualitative insights. The market size section will cover country-wide market size and forecast, which will include the impact of the COVID-19. The study further outlines the market segments, which are categorized into submarkets. The country-wise breakdowns are provided along with the size of the market. Competitive landscape includes the detailed assessment on companies in terms of product/service offerings, financial performance, overall business performance, and development strategies. The Europe gene editing market section of the report will b

In [154]:
j

0

In [165]:
one = set()
one.add("lamea")
one.add("italy")
one.add("france")
one.add("china")
one.add("brazil")
one.add("europe")
one.add("japan")
one.add("spain")
one.add("india")
one.add("uk")
one.add("germany")
one.add("mexico")
one.add("australia")
one.add("canada")
one.add("africa")
one.add("uae")
one.add("belgium")
one.add("switzerland")
one.add("netherlands")
one.add("thailand")
one.add("russia")
one.add("poland")
one.add("argentina")
one.add("singapore")
one.add("vietnam")
one.add("malaysia")
one.add("indonesia")
one.add("chile")
one.add("egypt")
one.add("eygpt")
one.add("algeria")
one.add("philippines")
one.add("turkey")
one.add("kenya")
one.add("sweden")
one.add("ireland")
one.add("venezuela")
one.add("nigeria")
one.add("norway")
one.add("taiwan")
one.add("hungary")
one.add("finland")
one.add("denmark")
one.add("tunisia")
one.add("ukraine")
one.add("qatar")
one.add("us")
one.add("slovakia")
one.add("itlay")
one.add("austria")

two = set()
two.add(("asia", "pacific"))
two.add(("south", "korea"))
two.add(("middle", "east"))
two.add(("u", "k"))
two.add(("u", "s"))
two.add(("north", "america"))
two.add(("saudi", "arabia"))
two.add(("south", "africa"))
two.add(("latin", "america"))
two.add(("united", "kingdom"))
two.add(("the", "netherlands"))
two.add(("new", "zealand"))
two.add(("south", "arabia"))

three = set()
three.add(("united", "arab", "emirates"))
three.add(("south", "east", "asia"))

for file_name in special_cases:
    words = file_name.split('-')
    if words[0] in one or tuple(words[:2]) in two or tuple(words[:3]) in three:
        continue
    print(file_name)

infrared-gas-sensor-market-A08273.pkl
connected-living-room-market-A107719.pkl
south-virtual-humans-market-A79541.pkl
neural-processor-market-A13155.pkl
banking-software-market-A109292.pkl
high-speed-camera-market.pkl
mobile-3d-market.pkl
intelligent-power-module-market.pkl
diabetes-therapeutics-market.pkl
top-std-testing-technologies-market.pkl
neoprene-fabric-market.pkl
flexible-oled-market-A06871.pkl
benelux-vegan-protein-powder-market-A89426.pkl
iran-islamic-republic-of-offshore-drilling-fluid-market-A93307.pkl
greece-business-travel-market-A109571.pkl
asean-cookware-market-A20277.pkl
consumer-electronics-packaging-market-A33330.pkl
asean-kids-toys-market-A101527.pkl
asean-freight-brokerage-market-A24747.pkl
gcc-brushless-dc-motors-market-A25233.pkl
zika-virus-vaccines-market.pkl
ksa-chlorinated-paraffin-market-A28408.pkl
asean-wearable-technology-market-A30926.pkl
construction-and-manufacturing-industry-market-A07192.pkl
colombia-modular-bamboo-market-A110021.pkl
vegan-ice-cream-m

In [162]:
from telegram import Bot

TELEGRAM_TOKEN = '6676215052:AAGgO_Z2WPubhsOAN2_KU7GAeXD9H82ZWw4'
CHAT_ID = '6625944042'

bot = Bot(token=TELEGRAM_TOKEN)

await bot.send_message(chat_id=CHAT_ID, text="done")

Message(channel_chat_created=False, chat=Chat(first_name='Shrikar', id=6625944042, type=<ChatType.PRIVATE>), date=datetime.datetime(2023, 10, 15, 23, 34, 58, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='Fragrantica-complete-pinger', id=6676215052, is_bot=True, username='Fragrantica_complete_bot'), group_chat_created=False, message_id=30, supergroup_chat_created=False, text='done')